In [10]:
import pickle
import sys
import numpy as np
import pandas as pd
import artm
import seaborn as sns
import matplotlib.pyplot as plt
print artm.version()

from os import path, mkdir
from datetime import datetime
sys.path.insert(0, '..\\modules\\helpers')

import distances_helper as dh 
import print_helper as ph
import create_model_helper as cmh
import build_convex_hull_helper as bchh
import different_models as dm

from plot_helper import PlotMaker
from config_helper import ConfigPaths
from scipy.optimize import minimize

0.8.1


In [11]:
config = ConfigPaths('config_sample_m3.cfg')
print config.models_file_name
models_file = open(config.models_file_name, 'a')

plot_maker = PlotMaker()

batch_vectorizer = artm.BatchVectorizer(data_path=config.output_batches_path,
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path=config.dictionary_path + '.dict')

Q:\\topic_modeling\\csi_science_collections.git\experiments\pn_model3\np_24_02_kl\models.txt


In [17]:
# permutations
def get_optimization_result_opt(dist_fn, phi_permuted, _phi_original, n_closest_topics_count):
    distances = bchh.calculate_distances(dist_fn, phi_permuted, _phi_original)
    opt_res_convex_hull = bchh.get_optimization_result(dist_fn, None, phi_permuted, _phi_original,
                                                             distances, n_closest_topics_count)
    return opt_res_convex_hull

def test_permutation_optimization(_phi_original, dist_fn, n_closest_topics_count, 
                                  get_optimization_result_fn, _debug_print=False):
    _phi_original = phi_original
    n_topics = _phi_original.shape[1]
    indices = np.random.choice(n_topics, n_topics, replace=False)
    phi_permuted = _phi_original.iloc[:, indices]
    opt_res_convex_hull = get_optimization_result_fn(dist_fn, phi_permuted, _phi_original, n_closest_topics_count)
    if _debug_print:
        for _, item in opt_res_convex_hull.items():
             print item['optimized_column'], item['fun'], item['x'][0:2], item['column_names'][0:2]
    matched_columns_count = 0
    good_x_coefficient_count = 0
    for _, item in opt_res_convex_hull.items():
        x_and_names = zip(item['x'], item['column_names'])
        x_and_names.sort(key=lambda x: x[0], reverse=True)
#         print x_and_names
        matched_columns_count += item['optimized_column'] == x_and_names[0][1]
        good_x_coefficient_count += x_and_names[0][0] > 0.98
    matched_columns = 100.0 * matched_columns_count / n_topics
    good_x_coefficient = 100.0 * good_x_coefficient_count / n_topics
    print 'matched_columns = {}%, good_x_coefficient = {}%'.format(matched_columns, good_x_coefficient)
    return matched_columns, good_x_coefficient

def test_combination_optimization(_phi_original, dist_fn, n_closest_topics_count, 
                                  get_optimization_result_fn, n_runs, _debug_print=False):
    _eps = 0.15
    not_found_count_list, matched_coeffs_count_list = [], []
    for n_iter in range(n_runs):               
        word_count, topics_count = phi_original.shape
        combination_count = np.random.randint(2, 7)
        combination_indices = np.random.choice(topics_count, combination_count, replace=False)
        phi_combination = phi_original.iloc[:, combination_indices]
        single_combination_weights = np.random.uniform(0, 1, combination_count)
        single_combination_weights = single_combination_weights / sum(single_combination_weights)
        combination_weights = np.array([single_combination_weights] * word_count)
        combination_weights = pd.DataFrame(data=combination_weights, index=phi_combination.index,
                                           columns=phi_combination.columns)
        phi_combination = phi_combination.multiply(combination_weights)
        phi_combination = pd.DataFrame(phi_combination.sum(axis=1), index=phi_combination.index, columns=['combination_0'])

        opt_res_convex_hull = get_optimization_result_fn(dist_fn, phi_combination, _phi_original, n_closest_topics_count)
        
        results = []
        for idx, col_name in enumerate(phi_original.columns[combination_indices]):
            opt_cols_names = list(opt_res_convex_hull['combination_0']['column_names'])
            col_name_opt_idx = opt_cols_names.index(col_name) if col_name in opt_cols_names else -1
            col_name_opt_x = opt_res_convex_hull['combination_0']['x'][col_name_opt_idx] if col_name_opt_idx != -1 else 0
            col_name_opt_idx = col_name_opt_idx if col_name_opt_idx != -1 else \
                               (0 if single_combination_weights[idx] < 0.1 else -1)
            results.append((col_name, single_combination_weights[idx], col_name_opt_x))           
        not_found_count = len([res for res in results if res[2] == -1]) * 100.0 / combination_count
        matched_coeffs_count = 100.0 * len([res for res in results if abs(res[1] - res[2]) < _eps]) / combination_count
        if _debug_print or not_found_count != 0 or matched_coeffs_count != 100: 
            print('it = {} / {} not_found_columns_count = {}%, matched_coeffs_count = {}%'.format(n_iter + 1, n_runs, 
                                                                            not_found_count, matched_coeffs_count))
            for r in results:
                print r
        not_found_count_list.append(not_found_count)
        matched_coeffs_count_list.append(matched_coeffs_count)
    total_not_found_count = sum(not_found_count_list) / n_runs
    total_matched_coeffs_count = sum(matched_coeffs_count_list) / n_runs
    print('total not_found_columns_count = {}%, matched_coeffs_count = {}%'.format(total_not_found_count,
                                                                                   total_matched_coeffs_count))
    return total_not_found_count, total_matched_coeffs_count

Загрузим оригинальный sample датасет (от model3), до этого скопировав в папку с batches нужные pickle файлы модели.
Сначала провизуалируем по одной итерации каждой новой модели, а потом будем итерационно строить выпуклую оболочку для каждой модели по отдельности и затем сравнивать их. 

In [13]:
phi_original, theta_original, saved_top_tokens_original, distances_hellinger_model_original = load_model_pickle('model3', 'distances_hellinger_model3', config.output_batches_path)
print phi_original.shape, theta_original.shape

(2216, 100) (100, 3446)


Запустить несколько раз с разным рандомом. Следить за тем, чтобы накапливались только независимые темы. Каждый раз смотреть. как проектируется на оригинальную матрицу.

# topic_modeling_coefficient

In [8]:
%matplotlib inline

In [9]:
#init model with phi
_debug_print = False
tm_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25)
(_, phi_ref) = tm_model.master.attach_model(model=tm_model.model_pwt)
if _debug_print:
    for model_description in tm_model.info.model:
        print model_description
np.copyto(phi_ref, phi_original.values) 

In [10]:
# test data
n_topics = phi_original.shape[1]
# indices = np.random.choice(n_topics, n_topics, replace=False)
# phi_permuted = phi_original.iloc[:, indices]
phi_permuted = phi_original
phi_permuted = phi_permuted.iloc[:, 0:10]

In [11]:
# create batch_vectorizer
vocabulary = {idx: word for idx, word in enumerate(phi_permuted.index)}
phi_values = np.array(phi_permuted.values, dtype=np.float)
test_batch_vectorizer = artm.BatchVectorizer(data_format='bow_n_wd',
                                             n_wd=phi_values,
                                             vocabulary=vocabulary)
# transform
theta_test = tm_model.transform(batch_vectorizer=test_batch_vectorizer)

In [15]:
X = phi_permuted

In [22]:
word_count, topics_count = phi_original.shape
combination_count = np.random.randint(2, 7)
combination_indices = np.random.choice(topics_count, combination_count, replace=False)
phi_combination = phi_original.iloc[:, combination_indices]
single_combination_weights = np.random.uniform(0, 1, combination_count)
single_combination_weights = single_combination_weights / sum(single_combination_weights)
combination_weights = np.array([single_combination_weights]*word_count)
combination_weights = pd.DataFrame(data=combination_weights, index=phi_combination.index, columns=phi_combination.columns)
phi_combination = phi_combination.multiply(combination_weights)
phi_combination = pd.DataFrame(phi_combination.sum(axis=1), index=phi_combination.index, columns=['combination_0'])


In [23]:
X = phi_combination

In [24]:
#init model with phi
_debug_print = False
tm_model = cmh.create_model(current_dictionary=dictionary, n_topics=100, n_doc_passes=5, seed_value=100,
                            n_top_tokens=15, p_mass_threshold=0.25)
(_, phi_ref) = tm_model.master.attach_model(model=tm_model.model_pwt)
if _debug_print:
    for model_description in tm_model.info.model:
        print model_description
np.copyto(phi_ref, phi_original.values) 

# test data
print X.shape, np.all(X.index == phi_original.index)

# create batch_vectorizer
vocabulary = {idx: word for idx, word in enumerate(X.index)}
X_values = np.array(X.values, dtype=np.float)
test_batch_vectorizer = artm.BatchVectorizer(data_format='bow_n_wd',
                                             n_wd=X_values,
                                             vocabulary=vocabulary)

# transform
theta_test = tm_model.transform(batch_vectorizer=test_batch_vectorizer)


(2216, 1) True


In [ ]:
print theta_test.shape
idx = 0
X_col = X.iloc[:, idx]
theta_test_col = theta_test.iloc[:, idx]
X_col_restated = phi_original.dot(theta_test_col)
print 'phi_orig = {}, theta_test_col = {}, X_restated = {}'.format(phi_original.shape, theta_test_col.shape, X_col_restated.shape)
is_close = np.all(abs(X_col_restated - X_col) < 1e-2)
print is_close, np.sum(abs(X_col_restated - X_col))

In [12]:
np.all(phi_original.columns == theta_test.index)

True

In [17]:
X_restated = phi_original.dot(theta_test)
print np.all(X_restated.index == X.index) 
print phi_original.shape, theta_test.shape, X_restated.shape, X.shape
is_close = np.all(abs(X_restated.values - X.values) < 0.001)
print is_close, np.sum(np.sum(abs(X_restated.values - X.values)))

True
(2216, 100) (100, 10) (2216, 10) (2216, 10)
False 18.3953


## inverse

In [14]:
# X = PHI_ORIGINAL * THETA
from numpy.linalg import inv, lstsq

In [18]:
def get_lsrsq_result(dist_fn, phi, phi_other, n_closest_topics_count):
    r = lstsq(phi_other, phi)
    x_values = r[0]
    res = {col: {'optimized_column': col, 'fun': dist_fn(phi[col], phi_other.dot(x_values[:, idx])), 
                 'column_names': phi_other.columns,
                 'x': x_values[:, idx]} for idx, col in enumerate(phi.columns)}
    return res

In [19]:
distance_fns = [dh.hellinger_dist, dh.kl_dist, dh.jaccard_dist, dh.cos_dist]
for dist_fn in distance_fns:
    print('Processing distance fn = {}'.format(dist_fn))
    _,_ = test_permutation_optimization(phi_original, dist_fn, n_closest_topics_count=5, 
                                        get_optimization_result_fn=get_lsrsq_result)
    _,_ = test_combination_optimization(phi_original, dist_fn, n_closest_topics_count=5, 
                                        get_optimization_result_fn=get_lsrsq_result,
                                        n_runs=100, _debug_print=False)

Processing distance fn = <function hellinger_dist at 0x000000000C457438>


..\modules\helpers\distances_helper.py:50: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / np.sqrt(2)


matched_columns = 100.0%, good_x_coefficient = 100.0%
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
Processing distance fn = <function kl_dist at 0x000000000C3DF438>
matched_columns = 100.0%, good_x_coefficient = 100.0%
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
Processing distance fn = <function jaccard_dist at 0x000000000C457208>
matched_columns = 100.0%, good_x_coefficient = 100.0%
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
Processing distance fn = <function cos_dist at 0x000000000C457358>
matched_columns = 100.0%, good_x_coefficient = 100.0%
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%


# em

In [50]:
EPS = 1e-30
INF = 1e+10
PERPLEXITY_EPS = 1.0

import time
import numpy.matlib as npm

def __init_matrix(height, width):
    result = np.random.rand(height, width)
    return result / np.sum(result, 0)

def count_perplexity(n_dw, Phi, Theta):
    value = 0.0
    for w in xrange(n_dw.shape[0]):
        p_wd = np.dot(Phi[w, :], Theta)
        value += np.sum(np.multiply(n_dw[w, :], np.nan_to_num(np.log(p_wd))))
    return np.exp(-1.0 / np.sum(np.sum(n_dw)) * value)

def plsa_em_vectorized_fixed_phi(n_dw, num_topics, Phi_init, max_iter=100, Theta_init=None, avoid_print=False):
    num_tokens, num_docs = n_dw.shape
    Phi = Phi_init
    Theta = Theta_init if not Theta_init is None else __init_matrix(num_topics, num_docs)

    old_perplexity_value = INF

    for i in xrange(max_iter):
        time_start = time.time()
        n_wt = np.zeros([num_tokens, num_topics]);
        Theta_new = np.copy(Theta)
        Z = np.dot(Phi, Theta)
        Z[Z == 0.0] = INF
        n_d = np.sum(n_dw, 0)
        n_d[n_d == 0.0] = INF
        Theta_new = np.divide(np.multiply(Theta, np.dot(np.transpose(Phi), np.divide(n_dw, Z))), npm.repmat(n_d, num_topics, 1))
        Theta = Theta_new

        perplexity_value = count_perplexity(n_dw, Phi, Theta)
        if not avoid_print:
            print 'Iter# {}, perplexity value: {}, elapsed time: {}'.format(i, perplexity_value,
                                                                            time.time() - time_start)
        if abs(perplexity_value - old_perplexity_value) < PERPLEXITY_EPS or i == (max_iter - 1):
            return Phi, Theta
        else:
            old_perplexity_value = perplexity_value

def get_em_result(dist_fn, phi, phi_other, n_closest_topics_count, _debug_print=False):
    phi_1, x_values = plsa_em_vectorized_fixed_phi(phi.values, num_topics=phi_other.shape[1], max_iter=40,
                                                  Phi_init=phi_other.values, Theta_init=None, avoid_print=True)
    X_restated = phi_other.values.dot(x_values)
    if _debug_print:
#         print X_restated.shape, phi_1.shape, phi_other.shape, phi.shape
        print 'phi close {}, is nan {}, restated close {} '.format(np.all(abs(phi_1 - phi_other) < 1e-4), 
                                            np.any(np.isnan(x_values)), 
                                            np.all(abs(X_restated - phi) < 1e-2), \
                                            np.sum(np.sum(abs(X_restated - phi))))
    res = {col: {'optimized_column': col, 'fun': dist_fn(phi[col], phi_other.dot(x_values[:, idx])), 
                 'column_names': phi_other.columns,
                 'x': x_values[:, idx]} for idx, col in enumerate(phi.columns)}
    return res

In [ ]:
phi_3, theta_3 = plsa_em_vectorized_fixed_phi(phi_permuted.values, num_topics=100, max_iter=10,
                                    Phi_init=phi_original.values, Theta_init=None, avoid_print=False)
X_restated = phi_original.values.dot(theta_3)
print np.all(abs(phi_3 - phi_original) < 1e-4), np.any(np.isnan(theta_3)), np.all(abs(X_restated - phi_permuted) < 1e-2), \
np.sum(np.sum(abs(X_restated - phi_permuted)))

In [51]:
# actually doesn't depend on metrics !!! (faster)
distance_fns = [dh.hellinger_dist, dh.kl_dist, dh.jaccard_dist, dh.cos_dist]
for dist_fn in distance_fns:
    print('Processing distance fn = {}'.format(dist_fn))
    _,_ = test_permutation_optimization(phi_original, dist_fn, n_closest_topics_count=5, 
                                        get_optimization_result_fn=get_em_result)
    print("-------------------------------------------------------")
    _,_ = test_combination_optimization(phi_original, dist_fn, n_closest_topics_count=5, 
                                        get_optimization_result_fn=get_em_result,
                                        n_runs=100, _debug_print=False)
    print("=======================================================")

Processing distance fn = <function hellinger_dist at 0x000000000C457438>


C:\Users\Sheldon\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in log


matched_columns = 100.0%, good_x_coefficient = 100.0%
-------------------------------------------------------
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
Processing distance fn = <function kl_dist at 0x000000000C3DF438>
matched_columns = 100.0%, good_x_coefficient = 99.0%
-------------------------------------------------------
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
Processing distance fn = <function jaccard_dist at 0x000000000C457208>
matched_columns = 100.0%, good_x_coefficient = 100.0%
-------------------------------------------------------
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%
Processing distance fn = <function cos_dist at 0x000000000C457358>
matched_columns = 100.0%, good_x_coefficient = 100.0%
-------------------------------------------------------
total not_found_columns_count = 0.0%, matched_coeffs_count = 100.0%


In [243]:
p = phi_original.iloc[:, 1]
q = phi_original.iloc[:, 2]
print dh.cos_dist(p, q), dh.kl_dist(p, q), dh.kl_sym_dist(p, q), dh.jaccard_dist(p, q)

# opt

In [43]:
def get_optimization(dist_fn, phi, phi_other, n_closest_topics_count):
    distances = bchh.calculate_distances(dist_fn, phi, phi_other)
    opt_res_convex_hull = bchh.get_optimization_result(dist_fn, None, phi, phi_other,
                                                       distances, n_closest_topics_count)
    return opt_res_convex_hull

In [46]:
distance_fns = [dh.hellinger_dist, dh.kl_dist, dh.jaccard_dist, dh.cos_dist]
for dist_fn in distance_fns:
    print('Processing distance fn = {}'.format(dist_fn))
    _,_ = test_permutation_optimization(phi_original, dist_fn, n_closest_topics_count=15, 
                                        get_optimization_result_fn=get_optimization)
    print("-------------------------------------------------------")
    _,_ = test_combination_optimization(phi_original, dist_fn, n_closest_topics_count=15, 
                                        get_optimization_result_fn=get_optimization,
                                        n_runs=100, _debug_print=False)
    print("=======================================================")

Processing distance fn = <function hellinger_dist at 0x000000000C457438>
Column topic_88 not optimized
Column topic_5 not optimized
Column topic_48 not optimized
Column topic_51 not optimized
Column topic_22 not optimized
Column topic_72 not optimized
Column topic_1 not optimized
Column topic_20 not optimized
Column topic_34 not optimized
Column topic_8 not optimized
Column topic_55 not optimized
Column topic_83 not optimized
Column topic_54 not optimized
Column topic_87 not optimized
Column topic_91 not optimized
Column topic_13 not optimized
Column topic_46 not optimized
Column topic_68 not optimized
Column topic_14 not optimized
Column topic_57 not optimized
Column topic_89 not optimized
Column topic_29 not optimized
Column topic_95 not optimized
Column topic_32 not optimized
Column topic_9 not optimized
Column topic_28 not optimized
Column topic_0 not optimized
Column topic_96 not optimized
Column topic_25 not optimized
Column topic_58 not optimized
Column topic_17 not optimized
Co